Task
- Join all three dataframes
- Start on EDA
- Model selection

Background Information on Dataset: BLAH BLAH BLAH

Importing Packages

In [25]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import matplotlib
import plotly
import plotly.express as px
import plotly.graph_objects as go


EDA

Load in dataset

Aside: Let's measure how long it takes to load in our dataset: using pandas and dask

In [2]:
import cProfile
import pstats

profiler = cProfile.Profile()
profiler.enable()
df = pd.read_parquet("Data/Large/articles.parquet")
profiler.disable()

profiler_stats = pstats.Stats(profiler)
profiler_stats.print_stats()

         10189 function calls (9867 primitive calls) in 2.432 seconds

   Random listing order was used

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 c:\Users\sulma\AppData\Local\Programs\Python\Python311\Lib\json\encoder.py:205(iterencode)
       11    0.000    0.000    0.000    0.000 c:\Users\sulma\AppData\Local\Programs\Python\Python311\Lib\enum.py:241(__set_name__)
        3    0.000    0.000    0.000    0.000 c:\Users\sulma\AppData\Local\Programs\Python\Python311\Lib\enum.py:192(__get__)
        2    0.000    0.000    0.000    0.000 c:\Users\sulma\AppData\Local\Programs\Python\Python311\Lib\json\encoder.py:183(encode)
        2    0.000    0.000    0.000    0.000 c:\Users\sulma\AppData\Local\Programs\Python\Python311\Lib\re\_compiler.py:386(<listcomp>)
        3    0.000    0.000    0.000    0.000 c:\Users\sulma\AppData\Local\Programs\Python\Python311\Lib\site-packages\_distutils_hack\__init__.py:96(<lambda>)

In [5]:
profiler = cProfile.Profile()
profiler.enable()
df = dd.read_parquet("Data/Large/articles.parquet", engine = "fastparquet")
profiler.disable()

profiler_stats = pstats.Stats(profiler)
profiler_stats.print_stats()

         32472 function calls (31941 primitive calls) in 0.174 seconds

   Random listing order was used

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1146    0.001    0.000    0.002    0.000 c:\Users\sulma\AppData\Local\Programs\Python\Python311\Lib\copy.py:66(copy)
        1    0.000    0.000    0.000    0.000 c:\Users\sulma\AppData\Local\Programs\Python\Python311\Lib\site-packages\zmq\sugar\socket.py:621(send)
      9/3    0.000    0.000    0.000    0.000 c:\Users\sulma\AppData\Local\Programs\Python\Python311\Lib\copy.py:128(deepcopy)
        2    0.000    0.000    0.000    0.000 c:\Users\sulma\AppData\Local\Programs\Python\Python311\Lib\enum.py:192(__get__)
        6    0.000    0.000    0.000    0.000 c:\Users\sulma\AppData\Local\Programs\Python\Python311\Lib\site-packages\_distutils_hack\__init__.py:96(<lambda>)
        1    0.000    0.000    0.000    0.000 c:\Users\sulma\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:757(__exit__)
     

c:\Users\sulma\AppData\Local\Programs\Python\Python311\Lib\site-packages\dask\dataframe\io\parquet\core.py:1241: FutureWarning: The fastparquet engine is deprecated and will be removed in a future release. Please install pyarrow.
  warnings.warn(


Even though the dataframe took 1.464 seconds to load using pandas, dask took 0.054 seconds. This is a huge speedup ~ 27x

In [2]:
#Load in various dataframes
## Articles
df_art = pd.read_parquet("Data/Large/articles.parquet")

## Behaviors
df_bev = pd.read_parquet("Data/Large/train/behaviors.parquet")

## History
df_his = pd.read_parquet("Data/Large/train/history.parquet")



Join the data sources

In [3]:
# Convert datatype of column first
df_bev['article_ids_clicked'] = df_bev['article_ids_clicked'].apply(lambda x: x[0])

In [4]:
# Join bevhaiors to article
df= df_bev.join(df_art.set_index("article_id"), on = "article_ids_clicked")

# Join bevhaiors to history 
df= df.join(df_his.set_index("user_id"), on = "user_id")

# Drop all other dataframes from me
df_bev = []
df_his = []
df_art = []

Visualizations

In [20]:
def plotly_index_values(groupby_object):
    """Takes in a clause from a pandas groupby statement and returns X and Y variables used for plotting"""

    index = groupby_object.index
    values = groupby_object.values

    return index, values



In [36]:
# What does the daily user activity look like? (Impression Time vs Count of users on that day)

# Problem is I need to get groupby user ids for every day and then increment the counter based on unique ids difficult (spend time on it after?)

## Convert datetime to date
df['impression_date'] = pd.to_datetime(df["impression_time"]).dt.date


## Group by
tmp = df.groupby(by = ['impression_date'])['user_id'].count()

## Get X, Y for plots
X, Y = plotly_index_values(groupby_object = tmp)

fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x =X, y = Y,
                         mode = 'lines+markers'
                         )

)

fig.show()


In [30]:
# What does the users' monthly activity look like?
df["impression_date"].unique()

array([datetime.date(2023, 5, 21), datetime.date(2023, 5, 22),
       datetime.date(2023, 5, 24), datetime.date(2023, 5, 20),
       datetime.date(2023, 5, 23), datetime.date(2023, 5, 25),
       datetime.date(2023, 5, 19), datetime.date(2023, 5, 18)],
      dtype=object)

In [52]:
pd.Timestamp(df["impression_time_fixed"][0]).date()

TypeError: Cannot convert input [['2023-04-28T17:39:09.000000' '2023-04-28T17:40:06.000000'
 '2023-04-28T19:04:20.000000' '2023-04-29T00:09:15.000000'
 '2023-04-29T00:09:56.000000' '2023-05-01T21:06:43.000000'
 '2023-05-01T23:34:20.000000' '2023-05-01T23:37:15.000000'
 '2023-05-01T23:37:53.000000' '2023-05-01T23:38:25.000000'
 '2023-05-01T23:38:53.000000' '2023-05-02T03:24:56.000000'
 '2023-05-03T01:18:24.000000' '2023-05-03T01:18:51.000000'
 '2023-05-03T01:19:09.000000' '2023-05-03T01:19:22.000000'
 '2023-05-03T01:19:52.000000' '2023-05-03T01:20:09.000000'
 '2023-05-03T01:20:26.000000' '2023-05-03T01:20:49.000000'
 '2023-05-03T01:21:11.000000' '2023-05-03T01:21:24.000000'
 '2023-05-03T01:21:41.000000' '2023-05-03T01:21:59.000000'
 '2023-05-03T01:22:22.000000' '2023-05-03T01:22:46.000000'
 '2023-05-03T01:23:16.000000' '2023-05-03T01:23:37.000000'
 '2023-05-03T01:25:54.000000' '2023-05-03T01:26:05.000000'
 '2023-05-03T01:26:59.000000' '2023-05-03T01:27:16.000000'
 '2023-05-03T01:27:30.0

In [53]:
df["impression_time_fixed"].apply(lambda dt: dt.date()).groupby([df.user_id, df.dat.apply(lambda dt: dt.year)]).nunique()

AttributeError: 'numpy.ndarray' object has no attribute 'date'

In [35]:
df

,impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,...,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed,impression_date
0,47727,NaN,2023-05-21 21:35:07,20.0,NaN,1,"[9482380, 9775183, 9744403, 9775297, 9774020, ...",9775183,18293,False,...,486176.0,116966.0,8601233.0,0.9877,Negative,"[2023-04-28T17:39:09.000000, 2023-04-28T17:40:...","[94.0, 54.0, 100.0, 100.0, 64.0, 32.0, 100.0, ...","[9741311, 9733782, 9741428, 9739272, 9741106, ...","[44.0, 12.0, 31.0, 32.0, 10.0, 1.0, 169.0, 32....",2023-05-21
1,47731,NaN,2023-05-21 21:32:33,13.0,NaN,1,"[9774557, 9774516, 9775331, 9775277, 9759966]",9759966,18293,False,...,366127.0,72064.0,4997750.0,0.8143,Negative,"[2023-04-28T17:39:09.000000, 2023-04-28T17:40:...","[94.0, 54.0, 100.0, 100.0, 64.0, 32.0, 100.0, ...","[9741311, 9733782, 9741428, 9739272, 9741106, ...","[44.0, 12.0, 31.0, 32.0, 10.0, 1.0, 169.0, 32....",2023-05-21
2,47736,NaN,2023-05-21 21:33:32,17.0,NaN,1,"[9759966, 9774557, 9775352, 9746360, 9772601, ...",9774652,18293,False,...,355008.0,82246.0,6088904.0,0.8485,Negative,"[2023-04-28T17:39:09.000000, 2023-04-28T17:40:...","[94.0, 54.0, 100.0, 100.0, 64.0, 32.0, 100.0, ...","[9741311, 9733782, 9741428, 9739272, 9741106, ...","[44.0, 12.0, 31.0, 32.0, 10.0, 1.0, 169.0, 32....",2023-05-21
3,47737,NaN,2023-05-21 21:38:17,27.0,NaN,1,"[9774580, 9775131, 9775202, 9774789, 9774972, ...",9775184,18293,False,...,203321.0,39827.0,2538445.0,0.7068,Neutral,"[2023-04-28T17:39:09.000000, 2023-04-28T17:40:...","[94.0, 54.0, 100.0, 100.0, 64.0, 32.0, 100.0, ...","[9741311, 9733782, 9741428, 9739272, 9741106, ...","[44.0, 12.0, 31.0, 32.0, 10.0, 1.0, 169.0, 32....",2023-05-21
4,47740,NaN,2023-05-21 21:36:02,48.0,NaN,1,"[9774826, 9775171, 9775076, 9769624, 9775056, ...",9774648,18293,False,...,373006.0,86758.0,6442733.0,0.8769,Negative,"[2023-04-28T17:39:09.000000, 2023-04-28T17:40:...","[94.0, 54.0, 100.0, 100.0, 64.0, 32.0, 100.0, ...","[9741311, 9733782, 9741428, 9739272, 9741106, ...","[44.0, 12.0, 31.0, 32.0, 10.0, 1.0, 169.0, 32....",2023-05-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12063885,580100982,NaN,2023-05-18 10:27:05,9.0,NaN,1,"[9142581, 9233208, 9345280, 9486080, 9769306, ...",9769306,2111769,False,...,1530146.0,239002.0,13315830.0,0.9677,Negative,"[2023-04-28T18:50:40.000000, 2023-04-28T18:51:...","[45.0, 100.0, 67.0, 14.0, 100.0, 100.0, 62.0, ...","[9741379, 9741032, 9743702, 9743207, 9741068, ...","[8.0, 57.0, 16.0, 9.0, 119.0, 24.0, 16.0, 41.0...",2023-05-18
12063886,580100984,NaN,2023-05-18 10:30:39,13.0,NaN,1,"[9440508, 9142581, 9486080, 9345280, 9720068]",9720068,2111769,False,...,564767.0,10821.0,667781.0,0.7884,Neutral,"[2023-04-28T18:50:40.000000, 2023-04-28T18:51:...","[45.0, 100.0, 67.0, 14.0, 100.0, 100.0, 62.0, ...","[9741379, 9741032, 9743702, 9743207, 9741068, ...","[8.0, 57.0, 16.0, 9.0, 119.0, 24.0, 16.0, 41.0...",2023-05-18
12063887,580100987,NaN,2023-05-18 10:27:28,13.0,NaN,1,"[9695098, 9345280, 9142581, 9747757, 9735909, ...",9735909,2111769,False,...,453605.0,80116.0,5717602.0,0.5229,Neutral,"[2023-04-28T18:50:40.000000, 2023-04-28T18:51:...","[45.0, 100.0, 67.0, 14.0, 100.0, 100.0, 62.0, ...","[9741379, 9741032, 9743702, 9743207, 9741068, ...","[8.0, 57.0, 16.0, 9.0, 119.0, 24.0, 16.0, 41.0...",2023-05-18
12063888,580100996,NaN,2023-05-18 10:08:18,35.0,NaN,2,"[9769917, 9771126, 9345280, 9233208, 9771166, ...",9771126,2111774,False,...,483915.0,102709.0,3475945.0,0.6922,Neutral,"[2023-04-27T12:28:42.000000, 2023-04-27T16:09:...","[nan, nan, 100.0, 100.0, 100.0, nan, 29.0, 100...","[9737266, 9739202, 9739200, 9739005, 9739065, ...","[10.0, 3.0, 10.0, 28.0, 23.0, 5.0, 8.0, 27.0, ...",2023-05-18


In [ ]:
# Testing 123

## OHBABY